In [1]:
import pandas as pd
import numpy as np
import requests, json
import time

In [2]:
file_path = "./data/"
df = pd.read_csv(f"{file_path}police_data.csv")

In [3]:
df = df[df['사건종류'] == '보이스피싱']

In [4]:
print(df.shape)
df.head()

(46268, 16)


,접수부서,접수완료일시,경찰청구분,사건상태,사건종류,신고자 성별,발생지점,발생지 위도,발생지 경도,동일사건여부,연도,월,연월,요일,시간대,일자
3846,충남청,2020-01-02 09:33:49,충남청,종결요청,보이스피싱,불상,충청남도 천안시 서북구 차암동(부성2동) 524,127.104436,36.841583,NaN,2020,1,2020-01,목,9,2020-01-02
3865,충남청,2020-01-02 09:42:45,충남청,종결요청,보이스피싱,불상,충청남도 아산시 풍기동(행정:온양6동) 404-33,127.021875,36.771208,NaN,2020,1,2020-01,목,9,2020-01-02
3960,충남청,2020-01-02 10:23:29,충남청,종결요청,보이스피싱,불상,충청남도 서산시 대산읍 대산리(행정:대산읍) 688-1,126.426100,36.947617,NaN,2020,1,2020-01,목,10,2020-01-02
4016,대전청,2020-01-02 10:46:31,대전청,종결요청,보이스피싱,불상,대전광역시 유성구 대정동(행정:진잠동) 1-4,127.336373,36.317773,NaN,2020,1,2020-01,목,10,2020-01-02
4049,충남청,2020-01-02 11:00:20,충남청,종결요청,보이스피싱,불상,충청남도 논산시 내동(행정:취암동) 1113,127.104068,36.182762,NaN,2020,1,2020-01,목,11,2020-01-02


In [5]:
df = df.dropna(subset='발생지 위도')

In [6]:
print(df.shape)
df.head()

(40458, 16)


,접수부서,접수완료일시,경찰청구분,사건상태,사건종류,신고자 성별,발생지점,발생지 위도,발생지 경도,동일사건여부,연도,월,연월,요일,시간대,일자
3846,충남청,2020-01-02 09:33:49,충남청,종결요청,보이스피싱,불상,충청남도 천안시 서북구 차암동(부성2동) 524,127.104436,36.841583,NaN,2020,1,2020-01,목,9,2020-01-02
3865,충남청,2020-01-02 09:42:45,충남청,종결요청,보이스피싱,불상,충청남도 아산시 풍기동(행정:온양6동) 404-33,127.021875,36.771208,NaN,2020,1,2020-01,목,9,2020-01-02
3960,충남청,2020-01-02 10:23:29,충남청,종결요청,보이스피싱,불상,충청남도 서산시 대산읍 대산리(행정:대산읍) 688-1,126.426100,36.947617,NaN,2020,1,2020-01,목,10,2020-01-02
4016,대전청,2020-01-02 10:46:31,대전청,종결요청,보이스피싱,불상,대전광역시 유성구 대정동(행정:진잠동) 1-4,127.336373,36.317773,NaN,2020,1,2020-01,목,10,2020-01-02
4049,충남청,2020-01-02 11:00:20,충남청,종결요청,보이스피싱,불상,충청남도 논산시 내동(행정:취암동) 1113,127.104068,36.182762,NaN,2020,1,2020-01,목,11,2020-01-02


In [7]:
df_get_address = df[df['발생지점'].isnull()]

In [8]:
df_get_address.shape

(3184, 16)

In [9]:
df_get_address.index

Int64Index([   4743,    4981,    5928,    7439,    7980,    8538,   11339,
              12040,   14001,   14191,
            ...
            3832203, 3832255, 3837482, 3837835, 3837861, 3841594, 3844052,
            3844356, 3844509, 3845853],
           dtype='int64', length=3184)

In [10]:
def get_address(lon, lat):
    url = f'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={lon}&y={lat}'
    headers = {"Authorization": "KakaoAK a94e4017bac532fb091adeb612471652"}
    api_json = requests.get(url, headers=headers)
    full_address = json.loads(api_json.text)

    return full_address

In [11]:
def reverse_geocode(lon, lat):
    # json request
    try:
        json_req = get_address(lon, lat)
        json_doc = json_req.get('documents')[0]
        json_address = json_doc.get('address_name')
    except:
        json_address = np.nan

    return json_address

In [12]:
get_address(df_get_address['발생지 위도'][4743], df_get_address['발생지 경도'][4743])

{'meta': {'total_count': 2},
 'documents': [{'region_type': 'B',
   'code': '3014010400',
   'address_name': '대전광역시 중구 중촌동',
   'region_1depth_name': '대전광역시',
   'region_2depth_name': '중구',
   'region_3depth_name': '중촌동',
   'region_4depth_name': '',
   'x': 127.41242388824838,
   'y': 36.33890461806434},
  {'region_type': 'H',
   'code': '3014056000',
   'address_name': '대전광역시 중구 중촌동',
   'region_1depth_name': '대전광역시',
   'region_2depth_name': '중구',
   'region_3depth_name': '중촌동',
   'region_4depth_name': '',
   'x': 127.41242388824838,
   'y': 36.33890461806434}]}

In [13]:
df_get_address.loc['발생지점'] = df_get_address.apply(lambda row : reverse_geocode(row['발생지 위도'], row['발생지 경도']), axis=1)

/var/folders/3q/0nv03n8j1vx538ntwhns580w0000gn/T/ipykernel_88970/3673787082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_get_address.loc['발생지점'] = df_get_address.apply(lambda row : reverse_geocode(row['발생지 위도'], row['발생지 경도']), axis=1)


In [14]:
df_get_address['발생지점'].isnull().sum()

3185

In [15]:
df_get_address['발생지점'].value_counts()

Series([], Name: 발생지점, dtype: int64)

In [16]:
df_get_address.head()

,접수부서,접수완료일시,경찰청구분,사건상태,사건종류,신고자 성별,발생지점,발생지 위도,발생지 경도,동일사건여부,연도,월,연월,요일,시간대,일자
4743,대전청,2020-01-02 15:09:28,대전청,종결요청,보이스피싱,불상,NaN,127.404663,36.341685,NaN,2020.0,1.0,2020-01,목,15.0,2020-01-02
4981,대전청,2020-01-02 16:26:52,대전청,종결요청,보이스피싱,불상,NaN,127.404663,36.341685,NaN,2020.0,1.0,2020-01,목,16.0,2020-01-02
5928,대전청,2020-01-02 21:58:07,대전청,종결요청,보이스피싱,불상,NaN,127.404663,36.341685,NaN,2020.0,1.0,2020-01,목,21.0,2020-01-02
7439,대전청,2020-01-03 11:06:07,대전청,종결요청,보이스피싱,불상,NaN,127.404663,36.341685,NaN,2020.0,1.0,2020-01,금,11.0,2020-01-03
7980,대전청,2020-01-03 14:19:24,대전청,종결요청,보이스피싱,불상,NaN,127.404663,36.341685,NaN,2020.0,1.0,2020-01,금,14.0,2020-01-03


In [17]:
df.shape

(40458, 16)

In [18]:
df = df.dropna(subset='발생지점')

In [19]:
df = pd.concat([df, df_get_address])

In [20]:
df[df.index == 4743]

,접수부서,접수완료일시,경찰청구분,사건상태,사건종류,신고자 성별,발생지점,발생지 위도,발생지 경도,동일사건여부,연도,월,연월,요일,시간대,일자
4743,대전청,2020-01-02 15:09:28,대전청,종결요청,보이스피싱,불상,NaN,127.404663,36.341685,NaN,2020.0,1.0,2020-01,목,15.0,2020-01-02


In [21]:
df[df['경찰청구분'] == '세종청']

,접수부서,접수완료일시,경찰청구분,사건상태,사건종류,신고자 성별,발생지점,발생지 위도,발생지 경도,동일사건여부,연도,월,연월,요일,시간대,일자
1178032,충남청,2020-12-01 09:46:22,세종청,종결요청,보이스피싱,여성,세종특별자치시 세종시 조치원읍 교리(행정:조치원읍) 129-1,127.296490,36.605644,NaN,2020.0,12.0,2020-12,화,9.0,2020-12-01
1178658,충남청,2020-12-01 15:55:57,세종청,종결요청,보이스피싱,여성,세종특별자치시 세종시 보람동(행정:보람동) 670,127.291076,36.477785,NaN,2020.0,12.0,2020-12,화,15.0,2020-12-01
1178703,충남청,2020-12-01 16:14:58,세종청,종결요청,보이스피싱,여성,세종특별자치시 세종시 조치원읍 침산리(행정:조치원읍) 257-1,127.292547,36.604826,NaN,2020.0,12.0,2020-12,화,16.0,2020-12-01
1179046,충남청,2020-12-01 19:39:40,세종청,종결요청,보이스피싱,남성,세종특별자치시 세종시 금남면 용포리(행정:금남면) 320,127.278120,36.462630,NaN,2020.0,12.0,2020-12,화,19.0,2020-12-01
1179090,충남청,2020-12-01 20:12:12,세종청,종결요청,보이스피싱,여성,세종특별자치시 세종시 금남면 용포리(행정:금남면) 139-2,127.281100,36.464470,NaN,2020.0,12.0,2020-12,화,20.0,2020-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3833446,충남청,2023-01-15 00:08:12,세종청,종결요청,보이스피싱,불상,세종특별자치시 세종시 조치원읍 남리(행정:조치원읍 ) 456,127.300360,36.594041,NaN,2023.0,1.0,2023-01,일,0.0,2023-01-15
3835507,충남청,2023-01-15 16:29:14,세종청,종결요청,보이스피싱,남성,세종특별자치시 세종시 소담동(한솔동) 333-10,127.296436,36.484548,NaN,2023.0,1.0,2023-01,일,16.0,2023-01-15
3844243,충남청,2023-01-18 10:51:29,세종청,종결요청,보이스피싱,남성,세종특별자치시 세종시 전동면 청송리(행정:전동면 ) 154-2,127.257393,36.664743,NaN,2023.0,1.0,2023-01,수,10.0,2023-01-18
3845654,충남청,2023-01-18 18:38:01,세종청,종결요청,보이스피싱,여성,세종특별자치시 세종시 연서면 봉암리(연서면 ) 54-89,127.284535,36.565027,Y,2023.0,1.0,2023-01,수,18.0,2023-01-18


In [22]:
df = df.dropna(subset='발생지점')

In [23]:
df.shape

(37274, 16)

In [24]:
df = df[df['발생지점'].str.contains("대전광역시|충청남도|세종특별자치시")]
df.shape

(35735, 16)

In [35]:
df['발생지점'][1178032].split(' ')

['세종특별자치시', '세종시', '조치원읍', '교리(행정:조치원읍)', '129-1']

In [42]:
sigungu_list = []
for idx in list(df.index):
    if df['발생지점'][idx].split(' ')[0] == '세종특별자치시':
        a = df['발생지점'][idx].split(' ')[2]
    else:
        a = df['발생지점'][idx].split(' ')[1]
    
    sigungu_list.append(a)

In [45]:
df['발생시군구'] = sigungu_list

In [46]:
df['발생시군구'].isnull().sum()

0

In [47]:
df.head()

,접수부서,접수완료일시,경찰청구분,사건상태,사건종류,신고자 성별,발생지점,발생지 위도,발생지 경도,동일사건여부,연도,월,연월,요일,시간대,일자,발생시도,발생시군구
3846,충남청,2020-01-02 09:33:49,충남청,종결요청,보이스피싱,불상,충청남도 천안시 서북구 차암동(부성2동) 524,127.104436,36.841583,NaN,2020.0,1.0,2020-01,목,9.0,2020-01-02,천안시,천안시
3865,충남청,2020-01-02 09:42:45,충남청,종결요청,보이스피싱,불상,충청남도 아산시 풍기동(행정:온양6동) 404-33,127.021875,36.771208,NaN,2020.0,1.0,2020-01,목,9.0,2020-01-02,아산시,아산시
3960,충남청,2020-01-02 10:23:29,충남청,종결요청,보이스피싱,불상,충청남도 서산시 대산읍 대산리(행정:대산읍) 688-1,126.426100,36.947617,NaN,2020.0,1.0,2020-01,목,10.0,2020-01-02,서산시,서산시
4016,대전청,2020-01-02 10:46:31,대전청,종결요청,보이스피싱,불상,대전광역시 유성구 대정동(행정:진잠동) 1-4,127.336373,36.317773,NaN,2020.0,1.0,2020-01,목,10.0,2020-01-02,유성구,유성구
4049,충남청,2020-01-02 11:00:20,충남청,종결요청,보이스피싱,불상,충청남도 논산시 내동(행정:취암동) 1113,127.104068,36.182762,NaN,2020.0,1.0,2020-01,목,11.0,2020-01-02,논산시,논산시


In [48]:
sido_list = []
for idx in list(df.index):
    if df['발생지점'][idx].split(' ')[0] == '세종특별자치시':
        a = df['발생지점'][idx].split(' ')[1]
    else:
        a = df['발생지점'][idx].split(' ')[0]
    
    sido_list.append(a)

df['발생시도'] = sido_list

In [49]:
df.head()

,접수부서,접수완료일시,경찰청구분,사건상태,사건종류,신고자 성별,발생지점,발생지 위도,발생지 경도,동일사건여부,연도,월,연월,요일,시간대,일자,발생시도,발생시군구
3846,충남청,2020-01-02 09:33:49,충남청,종결요청,보이스피싱,불상,충청남도 천안시 서북구 차암동(부성2동) 524,127.104436,36.841583,NaN,2020.0,1.0,2020-01,목,9.0,2020-01-02,충청남도,천안시
3865,충남청,2020-01-02 09:42:45,충남청,종결요청,보이스피싱,불상,충청남도 아산시 풍기동(행정:온양6동) 404-33,127.021875,36.771208,NaN,2020.0,1.0,2020-01,목,9.0,2020-01-02,충청남도,아산시
3960,충남청,2020-01-02 10:23:29,충남청,종결요청,보이스피싱,불상,충청남도 서산시 대산읍 대산리(행정:대산읍) 688-1,126.426100,36.947617,NaN,2020.0,1.0,2020-01,목,10.0,2020-01-02,충청남도,서산시
4016,대전청,2020-01-02 10:46:31,대전청,종결요청,보이스피싱,불상,대전광역시 유성구 대정동(행정:진잠동) 1-4,127.336373,36.317773,NaN,2020.0,1.0,2020-01,목,10.0,2020-01-02,대전광역시,유성구
4049,충남청,2020-01-02 11:00:20,충남청,종결요청,보이스피싱,불상,충청남도 논산시 내동(행정:취암동) 1113,127.104068,36.182762,NaN,2020.0,1.0,2020-01,목,11.0,2020-01-02,충청남도,논산시


In [50]:
file_name = 'voice_phising.csv'

In [51]:
df.to_csv(f'{file_path}{file_name}', index=False)